In [1]:
from osgeo import gdal
import numpy as np
import os
import glob

# Function definitions

In [2]:
from raster_utilities.utils.geotransform_calcs import CalculatePixelLims, CalculatePixelLims_GlobalRef

In [3]:
from raster_utilities.io.tiff_management import SaveLZWTiff, GetRasterProperties, ReadAOI_PixelLims

In [4]:
def ExtractAlignedSubImage(infile, outputDir, outputName, 
                           longitudeLims, latitudeLims,
                          maintainExtent = False, outNDV = None):
    '''
    Extracts and saves a sub-image from another image, given a lat/lon bounding box.
    
    The input bounding box should be specified in degrees.
    If maintainExtent=True then the output image will have the same extent 
    (size) as the original, with the clipped-out area set to nodata. 
    The output nodata value (outNDV) can be reset to something different from 
    the original, by default it will be passed through (this doesn't change the data 
    just the nodata tag, for cases where it is -9999 but not recorded as such)
    '''
    #existingRasterProps = GetRasterProperties(infile)
    inGT, inProj, inNDV, inWidth, inHeight, inRes, inDT = GetRasterProperties(infile)
    
    pixelLimsToRead = CalculatePixelLims(inGT, longitudeLims, latitudeLims)
    # = ((xmin, xmax), (ymin, ymax))
    arr, subsetGT, _, _ = ReadAOI_PixelLims(infile, pixelLimsToRead[0], pixelLimsToRead[1])
    if outNDV is None:
        ndv = inNDV
    else:
        ndv = outNDV
    if maintainExtent:
        SaveLZWTiff(arr, ndv, inGT, inProj, outputDir, outputName,
                   outShape=(inHeight, inWidth), 
                   outOffset=(pixelLimsToRead[1][0], pixelLimsToRead[0][0]))
    else:
        SaveLZWTiff(arr, ndv, subsetGT, inProj, outputDir, outputName)
    

In [5]:
def ExtractAlignedSubImageByPixels(infile, outputDir, outputName, 
                                  xLims, yLims, maintainExtent=False, outNDV=None):
    '''
    Extracts and saves a sub-image from another image, given pixel limit bounding box.
    
    The input bounding box should be specified in pixel offsets which are counted from an 
    origin of (0,0) at the top left corner of the image.
    If maintainExtent=True then the output image will have the same extent 
    (size) as the original, with the clipped-out area set to nodata. 
    The output nodata value (outNDV) can be reset to something different from 
    the original, by default it will be passed through (this doesn't change the data 
    just the nodata tag, for cases where it is -9999 but not recorded as such)
    '''
    inGT, inProj, inNDV, inWidth, inHeight, inRes, inDT = GetRasterProperties(infile)
    arr, subsetGT, _, _ = ReadAOI_PixelLims(infile, xLims, yLims)
    if outNDV is None:
        ndv = inNDV
    else: 
        ndv = outNDV
    if maintainExtent:
        SaveLZWTiff(arr, ndv, inGT, inProj, outputDir, outputName,
                   outShape=(inHeight, inWidth), 
                   outOffset=(yLims[0], xLims[0]))
    else:
        SaveLZWTiff(arr, ndv, subsetGT, inProj, outputDir, outputName)
    

#### check the limits calculations

In [ ]:
globalGT = rprop.gt
nonGlobalGT = (-120.0, 0.041666666666667, 0.0, 85.0, 0.0, -0.041666666666667)
reqXLims = (-120., 0.)
reqYLims = (60., -60)
realPixelCoords = CalculatePixelLims(globalGT, reqXLims, reqYLims)
forcedGlobalPixelCoords = CalculatePixelLims_GlobalRef(globalGT, reqXLims, reqYLims)
nonGlobalCoords = CalculatePixelLims(nonGlobalGT, reqXLims, reqYLims)

In [14]:
realPixelCoords

((1440, 4320), (720, 3600))

In [16]:
forcedGlobalPixelCoords

((1440, 4320), (720, 3600))

In [17]:
nonGlobalCoords

((0, 2880), (600, 3480))

# Usage

#### Clip a cube (folder) of images to lat limits varying by month, maintain input extent but replace with nodata

(we do this for the reflectance-based MODIS covariates as they're nonsense in high latitudes in winter (no daylight))

In [5]:
# the N-S limits we want by month. Derived empirically.
EVI_NS_Lims = {
    "01":(60,-60),
    "02":(68,-60),
    "03":(80,-60),
    "04":(80,-60),
    "05":(80,-60),
    "06":(80,-60),
    "07":(80,-60),
    "08":(80,-60),
    "09":(80,-60),
    "10":(68,-60),
    "11":(62,-60),
    "12":(60,-60)
}

In [6]:
extractFromDir = r'C:/temp/dataprep/modis/TCB_5KM_Aggregations/'
extractToDir = r'C:/temp/dataprep/modis/TCB_5KM_Aggregations_Clipped'

In [7]:
#inPattern = (os.path.join(extractFromDir,'*.tif'))
inPattern = (os.path.join(extractFromDir,'TCB*.tif'))
inFiles = glob.glob(inPattern)

In [8]:
# Clip each file to y limits varying according to the above list, don't clip x limits,
# use original filename but a different folder for output, and maintain the extent 
# so the results have the same dimensions but are set to nodata outside the limits
for infile in inFiles:
    mth = os.path.basename(infile).split('.')[2]
    latLims = EVI_NS_Lims[mth]
    ExtractAlignedSubImage(infile, extractToDir, 
                           longitudeLims=(-180, 180), latitudeLims=latLims, 
                           outputName=os.path.basename(infile),
                          maintainExtent=True)
    

#### Clip a folder of images to a fixed extent, output the new extent only

specify the extents in degrees, in the use_X and use_Y variables

In [7]:

#e8_X = (10,42)
#e8_Y = (-4,-36)
#khm_X = (102,108)
#khm_Y = (15,9)
moz_X = (30,41)
moz_Y = (-10,-27)

use_X = moz_X
use_Y = moz_Y

specify the files to process in the list variable inFiles

In [39]:
extractFromDir = r'H:\Night\Synoptic'
extractToDir = r'E:\Data\Harry\Documents\dial-a-map\moz_data'
#inPattern = (os.path.join(extractFromDir,'*.tif'))
inPattern = (os.path.join(extractFromDir,'*.mean.*.tif'))
inFiles = glob.glob(inPattern)
inFiles = [f for f in inFiles if int(os.path.basename(f).split('.')[1])==2013]

In [ ]:
inFiles

process all the files in turn, unless the output already exists

In [41]:
# Clip each file to x and y limits as specified above, use original filename but a 
# different folder for output, and output images with the new (clipped) extent
for infile in inFiles:
    outname = os.path.join(extractToDir, os.path.basename(infile))
    if os.path.exists(outname):
        continue
    ExtractAlignedSubImage(infile, extractToDir, 
                           longitudeLims=use_X, latitudeLims=use_Y, 
                           outputName=os.path.basename(infile),
                          maintainExtent=False)
    

#### process just one file

In [10]:
ExtractAlignedSubImage(r'G:/Supporting/CoastGlobal.tiff',
                      r'C:\Temp\dataprep\E8\Supporting',
                      longitudeLims=e8_X, latitudeLims=e8_Y,
                      outputName="Coast_E8.tif", maintainExtent=False)

In [10]:
ExtractAlignedSubImage(r'C:\Temp\EVI.2016.Annual.mean.5km.Mean.copy.tif',
                      r'C:\Temp',
                      longitudeLims=(43.016042,50.866771), latitudeLims=(-11.657728,-25.833370),
                      outputName='EVI_MDG.2016.Annual.mean.5km.Mean.tif',
                      maintainExtent=False)

#### Clip an image to a fixed pixel extent, i.e. trim a number of pixels off one or more sides

In [23]:
# trim the left and bottom-most pixels off an image we have previously checked to be 1682*1742
ExtractAlignedSubImageByPixels(r'inputfilepath',
                              r'outputfolder', 'Africa_MG_5K_template_clip.tif',
                              xLims=(1,1682), yLims=(0,1741))

#### Clip a folder tree of images to a fixed pixel extent, i.e. trim a number of pixels off one or more sides
Use os.walk rather than glob to handle folder structures more than one level deep

In [ ]:
# first check that everything in the tree is indeed the same pixel size
for root,dirs,files in os.walk(r'top\level\input\folder'):
    for f in files:
        if f.endswith('.tif'):
            pathname = os.path.join(root,f)
            props = GetRasterProperties(pathname)
            print (props.width, props.height)

In [21]:
# now run the command for each and mirror the folder structure below an output location
for root,dirs,files in os.walk(r'top\level\input\folder'):
    for f in files:
        if f.endswith('.tif'):
            pathname = os.path.join(root,f)
            props = GetRasterProperties(pathname)
            outdir = root.replace(r'top\level\input\folder',
                                 r'top\level\output\folder')
            ExtractAlignedSubImageByPixels(pathname,
                              outdir, f,
                              xLims=(1,1682), yLims=(0,1741))